# 1. Import and Install Dependencies

In [ ]:
!pip install opencv-python mediapipe scikit-learn matplotlib

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model): 
    #image = feed frame
    #model = Holistic model
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR -> RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB -> BGR
    return image, results

In [ ]:
#connection_drawing_spec 커스텀 (optional)
def draw_styled_landmarks(image, results):
    #Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values
results의 landmarks값 -> numpy array로 변환하기

In [ ]:
def extract_keypoints(results):
    # result의 landmarks의 모든 key point values -> 하나의 numpy array 로 flatten
    #if landmarks has no value, fill numpy array with zero
    
    if results.pose_landmarks: #pose landmarks
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() 
    else:
        pose = np.zeros(132) #33*4

    if results.left_hand_landmarks: #left hand landmarks
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() 
    else:
        lh = np.zeros(63) #21*3

    if results.right_hand_landmarks: #right hand landmarks
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() 
    else:
        rh = np.zeros(63) #21*3
    
    #return np.concatenate([pose, lh, rh])
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [ ]:
#pandas
!pip install xlrd
!pip install openpyxl
!pip install pandas
import pandas as pd

In [ ]:
#데이터 레이블 정보가 담긴 엑셀 파일
annotation_file_name = 'KETI-2018-SL-Annotation-v1.xlsx'

#엑셀 파일 데이터 프레임에 읽어오기
df = pd.read_excel(
    annotation_file_name, 
    sheet_name = 'KETI-2018-수어데이터-학습용-Annotation')

#단어 데이터만 필터링
df = df[df['타입(단어/문장)']=='단어']
#필요한 열만 추출
df = df.filter(items=['파일명', '한국어'], axis=1)
#모든 열 string 타입으로 변경
df = df.astype({'파일명':'string'},{'한국어':'string'})

print(df)

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_KSL_Entire_DataSet') 

#num of videos
no_sequences = 60

# 1 Video = 50 frames
sequence_length = 50

In [ ]:
for action in actions: #make directory for each actions
    for sequence in range(no_sequences): #make directory for each videos per action
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
##extract keypoints from video
def frame_capture(action, sequence, video):
    cap = cv2.VideoCapture(video)
    
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        frame_num = 0
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            if ret == False:
                break
                
            # Make detections
            image, results = mediapipe_detection(frame, holistic)  
            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen
            cv2.imshow('OpenCV Video', image)

            # export keypoints
            keypoints = extract_keypoints(results) #results of frame -> numpy array
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints) #save numpy array in directory
            
            #increase frame_num
            frame_num = frame_num + 1
                
            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
curDir = "C:/Users/User/Desktop/SCHOOL/2023-1/졸업 프로젝트/수어 데이터셋/최종 발표용/교통사고"
curAction = "accident"
curSequence = 0
for file in os.listdir(curDir):
    if(file.endswith(".mp4")):
        path = os.path.join(curDir, file)
        frame_capture(curAction, curSequence, path)
        curSequence = curSequence + 1

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
import fnmatch #폴더 안에 있는 npy 개수 계산하기 위함

#create two blank arrays
sequences, labels = [], [] 
#sequences(=videos) = feature data = x data 
#labels = label data = y data

for action in actions: 
    for sequence in range(no_sequences): #each action has 60 sequences(=videos)
        window = []
        
        #폴더 안에 있는 npy 개수(= frame 수) 계산
        file_count = len(fnmatch.filter(os.listdir(os.path.join(DATA_PATH, action, str(sequence))), '*.npy'))
        
        #저장된 npy 수가 원하는 sequence_length보다 같거나 많을 경우
        if(file_count >= sequence_length):  
            #앞에 있는 npy 버리고, start부터 끝까지 sequence_length개의 npy 가져오기
            start = file_count - sequence_length
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(start + frame_num)))
                window.append(res)
        #저장된 npy 수가 원하는 sequence_length보다 적을 경우
        else:
            #앞에 0으로 채운 npy 추가하기
            for frame_num in range(sequence_length - file_count):
                res = np.zeros(258) #132 + 63 + 63
                window.append(res)
            for frame_num in range(file_count):
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
                              
        #map label
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int) 
#converted label from int to binary array
#1 -> [1,0,0], 2 -> [0,1,0], 3 -> [0,0,1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
#split x,y data into train data and test data
#train data 95%, test data size 5% 

# 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard 

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
#tensor board is used to monitor neural network training and it's accuracy

In [ ]:
#build neural network architecture
model = Sequential()
drop_rate = 0.3

#add 3 lstm layers
model.add(LSTM(512, return_sequences=True, activation='tanh', input_shape=(50,258)))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(LSTM(256, return_sequences=True, activation='tanh'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(tf.keras.layers.Dropout(drop_rate))
#64 lstm units, set return_sequences = false (means end of lstm layers)

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
predict_res = model.predict(X_test)

In [ ]:
#y_test의 값은 각 label에 해당할 확률 값 ex.[0.1, 0.2, 0.7]
#y_test의 값이 가리키는 label 값 = 가장 큰 값을 가진 값
actions[np.argmax(predict_res[3])]

In [ ]:
#y_test의 값은 binary array 형식 ex.[0, 0, 1]
#y_test의 값이 가리키는 label 값 = 1을 가진 값
actions[np.argmax(y_test[3])]

# 9. Save Weights

In [ ]:
#단일 file/folder 에 모델의 아키텍처, 가중치 및 훈련 구성을 저장
model.save('kslaction.h5')

In [ ]:
del model

In [ ]:
model.load_weights('kslaction_50_frame.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
#y_test의 값들이 가리키는 label들을 하나의 리스트로 변환
#ex. [[0, 0, 1], [0, 1, 0]] -> [2, 1]
ytrue = np.argmax(y_test, axis=1).tolist()
#yhat의 값들이 가리키는 label들을 하나의 리스트로 변환
#ex. [[0.1, 0.2, 0.7], [0.1, 0.8, 0.1]] -> [2, 1]
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
#returns a confusion matrix sorted by the label order
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11-0. Translation

In [ ]:
import urllib.request
import json

In [ ]:
#papago translation api
url = 'https://openapi.naver.com/v1/papago/n2mt'
client_id = 'w7qb8_wV1jeo3Mx7iMEg'
client_secret = 'CqEcD1gtTd'

source = 'ko'
target = 'en'

def translate(text, target):
    encText = urllib.parse.quote(text)
    data = f'source={source}&target={target}&text=' + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if (rescode == 200):
        response_body = response.read()
        decode = json.loads(response_body.decode('utf-8'))
        result = decode['message']['result']['translatedText']
        return result
    else:
        print("Error Code:" + rescode)
        return "Error Code:" + rescode

# 11-1. Test using video

In [ ]:
# prediction using video
def video_prediction(video):
    
    # 1. detection variables
    sequence = [] #collect 60 frames to make a sequence(=video)
    sentence = [] #concatenate history of predictions together
    threshold = 0.7

    cap = cv2.VideoCapture(video)
    
    #clear subtitles when {clear_cycle} frames passed without new subtitle added
    frames_without_new_subtitle = 0
    clear_cycle = 300
    
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            # Read video
            ret, frame = cap.read()
            if ret == False:
                break
                
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-60:] #generate sequence with last 30 frames
            
            frames_without_new_subtitle += 1

            if len(sequence) == 60:
                #sequence.shape = (60, 1662)
                #the input shape model expects = (number of sequences, 60, 1662)
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])

                #3. Rendering logic
                #ex. res = [0.1, 0.2, 0.7]
                #np.argmax(res) = 2, res[np.argmax(res)] = 0.7
                if res[np.argmax(res)] > threshold: 
                    #new subtitle added
                    frames_without_new_subtitle = 0

                    cur_action_korean = actions_korean[np.argmax(res)]

                    if len(sentence) > 0: 
                        #sentence에 저장된 prediction이 있는 경우 
                        #새로운 prediction인 경우에만 sentence에 추가
                        if cur_action_korean != sentence[-1]:
                            sentence.append(cur_action_korean)
                    else: 
                        #sentence에 저장된 prediction 없는 경우 바로 sentence에 추가
                        sentence.append(cur_action_korean)

                    #sentence가 너무 길어지지 않도록 마지막 5개의 prediction만 유지
                    if len(sentence) > 5: 
                        sentence = sentence[-5:]
                
                #Clear subtitles if needed
                if frames_without_new_subtitle >= clear_cycle:
                    sentence.clear()
                
                #Render subtitles
                cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1) 
                if target == 'ko':
                    #putKoreanText(src, text, pos, font_size, font_color
                    image = putKoreanText(image, ' '.join(sentence), (3,10), 20, (255, 255, 255))
                else:
                    cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

                # Show to screen
                cv2.imshow('Video Prediction', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
curDir = "C:/Users/User/Desktop/SCHOOL/2023-1/졸업 프로젝트/수어 데이터셋/최종 발표용/아랫집에불이났어요"
for file in os.listdir(curDir):
    if(file.endswith(".mp4")):
        path = os.path.join(curDir, file)
        video_prediction(path)

# 11-2. Test in Real Time

In [ ]:
actions_korean = ['집', '도둑', '숨이 안쉬어져요', '아래', '불이 났어요', '낯선사람', '교통사고']

In [ ]:
#한글 텍스트 출력
from PIL import ImageFont, ImageDraw, Image

def putKoreanText(src, text, pos, font_size, font_color):
    img_pil = Image.fromarray(src)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype('C:/Users/User/ActionDetectionforSignLanguage/fonts/gulim.ttc', font_size)
    draw.text(pos, text, font=font, fill= font_color)
    return np.array(img_pil)


In [ ]:
# 1. detection variables
sequence = [] #collect 60 frames to make a sequence(=video)
sentence = [] #concatenate history of predictions together
threshold = 0.7


cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-50:] #generate sequence with last 30 frames
        
        if len(sequence) == 50:
            #sequence.shape = (60, 1662)
            #the input shape model expects = (number of sequences, 60, 1662)
            res = model.predict(np.expand_dims(sequence, axis=0))[0] #predict one sequence at a time
            print(actions[np.argmax(res)])
            
            #3. Rendering logic
            #ex. res = [0.1, 0.2, 0.7]
            #np.argmax(res) = 2, res[np.argmax(res)] = 0.7
            if res[np.argmax(res)] > threshold: 
                cur_action_korean = actions_korean[np.argmax(res)]
                
                if len(sentence) > 0: 
                    #sentence에 저장된 prediction이 있는 경우 
                    #새로운 prediction인 경우에만 sentence에 추가
                    if cur_action_korean != sentence[-1]:
                        sentence.append(cur_action_korean)
                else: 
                    #sentence에 저장된 prediction 없는 경우 바로 sentence에 추가
                    sentence.append(cur_action_korean)
            
            #sentence가 너무 길어지지 않도록 마지막 5개의 prediction만 유지
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            #Visualize probabilities
            #image = prob_viz(res, actions, image, colors)
            
            #cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1) 
            
            #cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
            #putKoreanText(src, text, pos, font_size, font_color
            image = putKoreanText(image, ' '.join(sentence), (3,10), 20, (255, 255, 255))
            
           
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()